# IHLT Project


In [14]:
# Autoreload
%load_ext autoreload
%autoreload 2

import pandas as pd
import csv
import nltk

from utils.preprocessor import Preprocessor
from features.dummy_features import add_dummy_features
from features.statistical_features import add_POS_statistics, add_synset_statistics, add_synset_statistics_ext

from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
nltk.download('punkt_tab')
nltk.download('stopwords')



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\paubl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\paubl\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\paubl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\paubl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\paubl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Load data

In [9]:
TRAIN_PATH = '../data/train/01_raw/'
TRAIN_GS_PATH = '../data/train/scores/'
TEST_PATH = '../data/test/01_raw/'
TEST_GS_PATH = '../data/test/scores/'

def load_data(path_f, path_gs, files):
    # Read first file
    dt = pd.read_csv(path_f + 'STS.input.' + files[0] + '.txt', sep='\t', quoting=csv.QUOTE_NONE, header=None, names=['s1', 's2'])
    dt['gs'] = pd.read_csv(path_gs + 'STS.gs.' + files[0] + '.txt', sep='\t', header=None, names=['gs'])
    # Concatenate the rest of files
    for f in files[1:]:
        dt2 = pd.read_csv(path_f + 'STS.input.' + f + '.txt', sep='\t', quoting=csv.QUOTE_NONE, header=None, names=['s1', 's2'])
        dt2['gs'] = pd.read_csv(path_gs + 'STS.gs.' + f + '.txt', sep='\t', header=None, names=['gs'])
        dt = pd.concat([dt, dt2], ignore_index=True)
    return dt

## Create models

In [10]:
def train_NN(df, input, output):
    X = df[input]
    y = df[output]
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
    model = Sequential()
    model.add(Dense(10, input_dim=len(input), activation='relu'))
    model.add(Dense(5, activation='relu'))               
    model.add(Dense(1))                                   
    model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32)
    return history, model


In [11]:
def plot_history(history):
    import matplotlib.pyplot as plt
    plt.plot(history.history['mae'])
    plt.plot(history.history['val_mae'])
    plt.title('model mae')
    plt.ylabel('mae')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

## Basic pipeline

### Train a model

In [ ]:
# Create the desired features

def add_features(dt):
    #add_dummy_features(dt)
    add_POS_statistics(dt)
    add_synset_statistics_ext(dt)

# Load train data
all_train_files = ['SMTeuroparl', 'MSRvid', 'MSRpar']
df = load_data(TRAIN_PATH, TRAIN_GS_PATH, all_train_files)

# Add features to the train data
add_features(df)

# Print all columns of df
print(df.columns)

# Features to use in the model
features = ['dif_n_words', 'dif_n_verbs', 'dif_n_nouns', 'dif_n_adjectives', 'dif_n_adverbs', 
            
            'all_all_shared_synsets_count', 'all_all_shared_synsets_ratio', 'all_all_avg_synset_similarity', 'all_all_max_synset_similarity',
            'all_verb_shared_synsets_count', 'all_verb_shared_synsets_ratio', 'all_verb_avg_synset_similarity', 'all_verb_max_synset_similarity',
            'all_noun_shared_synsets_count', 'all_noun_shared_synsets_ratio', 'all_noun_avg_synset_similarity', 'all_noun_max_synset_similarity',
            'all_adj_shared_synsets_count', 'all_adj_shared_synsets_ratio', 'all_adj_avg_synset_similarity', 'all_adj_max_synset_similarity',
            'all_adv_shared_synsets_count', 'all_adv_shared_synsets_ratio', 'all_adv_avg_synset_similarity', 'all_adv_max_synset_similarity',

            'best_all_shared_synsets_count', 'best_all_shared_synsets_ratio', 'best_all_avg_synset_similarity', 'best_all_max_synset_similarity',
            'best_verb_shared_synsets_count', 'best_verb_shared_synsets_ratio', 'best_verb_avg_synset_similarity', 'best_verb_max_synset_similarity',
            'best_noun_shared_synsets_count', 'best_noun_shared_synsets_ratio', 'best_noun_avg_synset_similarity', 'best_noun_max_synset_similarity',
            'best_adj_shared_synsets_count', 'best_adj_shared_synsets_ratio', 'best_adj_avg_synset_similarity', 'best_adj_max_synset_similarity',
            'best_adv_shared_synsets_count', 'best_adv_shared_synsets_ratio', 'best_adv_avg_synset_similarity', 'best_adv_max_synset_similarity',
            ]

# # Train the model
hist, model = train_NN(df, features, 'gs')
plot_history(hist)

Processed 2234/2234 rows (100.0%)%)
Index(['s1', 's2', 'gs', 'dif_n_words', 'dif_n_verbs', 'dif_n_nouns',
       'dif_n_adjectives', 'dif_n_adverbs', 'all_all_shared_synsets_count',
       'all_all_shared_synsets_ratio', 'all_all_avg_synset_similarity',
       'all_all_max_synset_similarity', 'all_verb_shared_synsets_count',
       'all_verb_shared_synsets_ratio', 'all_verb_avg_synset_similarity',
       'all_verb_max_synset_similarity', 'all_noun_shared_synsets_count',
       'all_noun_shared_synsets_ratio', 'all_noun_avg_synset_similarity',
       'all_noun_max_synset_similarity', 'all_adj_shared_synsets_count',
       'all_adj_shared_synsets_ratio', 'all_adj_avg_synset_similarity',
       'all_adj_max_synset_similarity', 'all_adv_shared_synsets_count',
       'all_adv_shared_synsets_ratio', 'all_adv_avg_synset_similarity',
       'all_adv_max_synset_similarity', 'best_all_shared_synsets_count',
       'best_all_shared_synsets_ratio', 'best_all_avg_synset_similarity',
       'best_al

KeyError: "['bestv_verb_avg_synset_similarity'] not in index"

## Compute correlation of the model

In [ ]:
from scipy.stats import pearsonr

# Load test data
# Doubt: The files with the same name are the same files as in the training set? If so, we have to use them also for testing?
all_test_files = ['SMTeuroparl', 'MSRvid', 'MSRpar', 'surprise.OnWN', 'surprise.SMTnews']
df_test = load_data(TEST_PATH, TEST_GS_PATH, all_test_files)

# Add the features to the test data
add_features(df_test)
# Fill column of the dataset with the predictions of the model
df_test['predicted'] = model.predict(df_test[features])

# Compute the Pearson correlation between the predictions and the gold standard
corr = pearsonr(df_test['gs'], df_test['predicted'])[0]
print('Pearson correlation:', corr)


Processed 3108/3108 rows (100.0%)%)
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 995us/step
Pearson correlation: 0.6482390590064078


: 